### Pizza sales data analyzed with both SQL and Python and a dashboard using Power BI

In [1]:
#!pip install ipython-sql prettytable


In [2]:
import sqlite3
import prettytable
import seaborn
import pandas as pd


import warnings
warnings.filterwarnings('ignore')

/Users/sinegugumthembu/anaconda3/envs/AI4WSA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [17]:
df = pd.read_csv('pizza_sales.csv')
#df.columns
df.head(3)

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,01-01-2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,01-01-2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,01-01-2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza


In [70]:
df['order_date']=pd.to_datetime(df['order_date'],format='%d-%m-%Y')
df.head(3)

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza


In [71]:
conn =sqlite3.connect('pizza_sales.db')
cursor =conn.cursor()

In [95]:
# Create the pizza_sales table

create_table = '''

CREATE TABLE IF NOT EXISTS pizza_sales (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pizza_id INTEGER,
    order_id INTEGER,
    pizza_name_id TEXT,
    quantity INTEGER,
    order_date DATE,
    order_time TEXT,
    unit_price REAL,
    total_price REAL,
    pizza_size TEXT,
    pizza_category TEXT,
    pizza_ingredients TEXT,
    pizza_name TEXT);
'''
cursor.execute(create_table)

In [96]:
df.to_sql('pizza_sales', conn, if_exists='replace', index=False)

48620

In [97]:
view_db = "SELECT * FROM pizza_sales LIMIT 10;"
cursor.execute(view_db)
result = pd.read_sql_query(view_db, conn)
print(result)

   pizza_id  order_id  pizza_name_id  quantity           order_date  \
0         1         1     hawaiian_m         1  2015-01-01 00:00:00   
1         2         2  classic_dlx_m         1  2015-01-01 00:00:00   
2         3         2  five_cheese_l         1  2015-01-01 00:00:00   
3         4         2    ital_supr_l         1  2015-01-01 00:00:00   
4         5         2     mexicana_m         1  2015-01-01 00:00:00   
5         6         2     thai_ckn_l         1  2015-01-01 00:00:00   
6         7         3    ital_supr_m         1  2015-01-01 00:00:00   
7         8         3   prsc_argla_l         1  2015-01-01 00:00:00   
8         9         4    ital_supr_m         1  2015-01-01 00:00:00   
9        10         5    ital_supr_m         1  2015-01-01 00:00:00   

  order_time  unit_price  total_price pizza_size pizza_category  \
0   11:38:36       13.25        13.25          M        Classic   
1   11:57:40       16.00        16.00          M        Classic   
2   11:57:40     

In [98]:
sum_total_price_q = "SELECT SUM(total_price) AS total_revenue FROM pizza_sales;"
cursor.execute(sum_total_price_q)
sum_total_price = pd.read_sql_query(sum_total_price_q, conn)
print('The total revenue is:', sum_total_price.loc[0])


The total revenue is: total_revenue    817860.05
Name: 0, dtype: float64


In [77]:
avg_price_order_q = "SELECT SUM(total_price)/ COUNT (DISTINCT order_id) AS avg_price_order FROM pizza_sales;"
cursor.execute(avg_price_order_q)
avg_price_order = pd.read_sql_query(avg_price_order_q, conn)
print('The average price per order is:', avg_price_order.loc[0])

The average price per order is: avg_price_order    38.307262
Name: 0, dtype: float64


In [78]:
num_total_orders_q = "SELECT COUNT (DISTINCT order_id) AS num_total_orders FROM pizza_sales;"
cursor.execute(num_total_orders_q)
num_total_orders = pd.read_sql_query(num_total_orders_q, conn)
print('The total number of orders is:', num_total_orders.loc[0])

The total number of orders is: num_total_orders    21350
Name: 0, dtype: int64


In [79]:
total_pizzas_q = "SELECT SUM(quantity) AS total_pizzas FROM pizza_sales;"
cursor.execute(total_pizzas_q)
total_pizzas = pd.read_sql_query(total_pizzas_q, conn)
print('The number of pizzas per order is:', total_pizzas.loc[0])

The number of pizzas per order is: total_pizzas    49574
Name: 0, dtype: int64


In [80]:
#avg_pizzas_sold_q = "SELECT CAST(SUM(quantity) AS DECIMAL(10,2))/COUNT (DISTINCT order_id) AS avg_pizzas_sold FROM pizza_sales;"
avg_pizzas_sold_q = "SELECT SUM(quantity) /COUNT (DISTINCT order_id) AS avg_pizzas_sold FROM pizza_sales;"
cursor.execute(avg_pizzas_sold_q)
avg_pizzas_sold = pd.read_sql_query(avg_pizzas_sold_q, conn)
print('The average number of pizzas sold is:', avg_pizzas_sold.loc[0])

The average number of pizzas sold is: avg_pizzas_sold    2
Name: 0, dtype: int64


In [87]:
total_orders_per_day_q = "SELECT strftime('%w', order_date) AS order_day, COUNT(DISTINCT order_id) AS total_orders_per_day FROM pizza_sales GROUP BY strftime('%w', order_date);"
cursor.execute(total_orders_per_day_q)
total_orders_per_day = pd.read_sql_query(total_orders_per_day_q, conn)
print(total_orders_per_day)

  order_day  total_orders_per_day
0         0                  2624
1         1                  2794
2         2                  2973
3         3                  3024
4         4                  3239
5         5                  3538
6         6                  3158


In [88]:
# Map day numbers to day names
day_mapping = {
    '0': 'Sunday',
    '1': 'Monday',
    '2': 'Tuesday',
    '3': 'Wednesday',
    '4': 'Thursday',
    '5': 'Friday',
    '6': 'Saturday'
}
# Replace day numbers with day names
total_orders_per_day['order_day'] = total_orders_per_day['order_day'].map(day_mapping)
# Print the result with day names
print(total_orders_per_day)

   order_day  total_orders_per_day
0     Sunday                  2624
1     Monday                  2794
2    Tuesday                  2973
3  Wednesday                  3024
4   Thursday                  3239
5     Friday                  3538
6   Saturday                  3158


In [91]:
total_orders_per_month_q = "SELECT strftime('%m', order_date) AS order_month, COUNT(DISTINCT order_id) AS total_orders_per_month FROM pizza_sales GROUP BY strftime('%m', order_date) ORDER BY total_orders_per_month DESC;"
cursor.execute(total_orders_per_month_q)
total_orders_per_month = pd.read_sql_query(total_orders_per_month_q, conn)
print(total_orders_per_month)

   order_month  total_orders_per_month
0           07                    1935
1           05                    1853
2           01                    1845
3           08                    1841
4           03                    1840
5           04                    1799
6           11                    1792
7           06                    1773
8           02                    1685
9           12                    1680
10          09                    1661
11          10                    1646


In [90]:
total_orders_per_month = pd.read_sql_query(total_orders_per_month_q, conn)
# Map month numbers to month names
month_mapping = {
    '01': 'January',
    '02': 'February',
    '03': 'March',
    '04': 'April',
    '05': 'May',
    '06': 'June',
    '07': 'July',
    '08': 'August',
    '09': 'September',
    '10': 'October',
    '11': 'November',
    '12': 'December'
}
# Replace month numbers with month names
total_orders_per_month['order_month'] = total_orders_per_month['order_month'].map(month_mapping)
# Print the result with month names
print(total_orders_per_month)

   order_month  total_orders_per_month
0      January                    1845
1     February                    1685
2        March                    1840
3        April                    1799
4          May                    1853
5         June                    1773
6         July                    1935
7       August                    1841
8    September                    1661
9      October                    1646
10    November                    1792
11    December                    1680


In [100]:
percentage_sale_category_q = "SELECT pizza_category, SUM(total_price)*100/(SELECT SUM(total_price) FROM pizza_sales) FROM pizza_sales AS total_per_category GROUP BY pizza_category;"
cursor.execute(percentage_sale_category_q)
percentage_sale_category = pd.read_sql_query(percentage_sale_category_q, conn)
print(percentage_sale_category)

  pizza_category  \
0        Chicken   
1        Classic   
2        Supreme   
3         Veggie   

   SUM(total_price)*100/(SELECT SUM(total_price) FROM pizza_sales)  
0                                          23.955138                
1                                          26.905960                
2                                          25.456311                
3                                          23.682591                


In [108]:
percentage_sale_permonth_category_q = """SELECT 
    pizza_category, 
    SUM(total_price) AS total_sales, 
    (SUM(total_price) * 100.0 / (SELECT SUM(total_price) FROM pizza_sales WHERE strftime('%m', order_date) = '01')) AS percentage_sales
FROM 
    pizza_sales
WHERE 
    strftime('%m', order_date) = '01'
GROUP BY 
    pizza_category;
"""
cursor.execute(percentage_sale_permonth_category_q)
total_permonth_category = pd.read_sql_query(percentage_sale_permonth_category_q, conn)
print(total_permonth_category)

  pizza_category  total_sales  percentage_sales
0        Chicken     16188.75         23.195278
1        Classic     18619.40         26.677919
2        Supreme     17929.75         25.689787
3         Veggie     17055.40         24.437016


In [109]:
percentage_sale_permonth_category_q = """SELECT 
    pizza_size, 
    SUM(total_price) AS total_sales, 
    (SUM(total_price) * 100.0 / (SELECT SUM(total_price) FROM pizza_sales WHERE strftime('%m', order_date) = '01')) AS percentage_sales
FROM 
    pizza_sales
WHERE 
    strftime('%m', order_date) = '01'
GROUP BY 
    pizza_size;
"""
cursor.execute(percentage_sale_permonth_category_q)
total_permonth_category = pd.read_sql_query(percentage_sale_permonth_category_q, conn)
print(total_permonth_category)

  pizza_size  total_sales  percentage_sales
0          L      32399.4         46.421934
1          M      20943.5         30.007895
2          S      15103.5         21.640329
3         XL       1275.0          1.826823
4        XXL         71.9          0.103018


In [117]:
# SQL query to calculate percentage sales per pizza size for each pizza category

percentage_size_permonth_category_q = """

SELECT  
    pizza_size,
    SUM(total_price) AS total_sales, 
    (SUM(total_price) * 100.0 / (SELECT SUM(total_price) FROM pizza_sales)) AS percentage_sales
FROM 
    pizza_sales
GROUP BY 
     pizza_size;
"""

total_per_size = pd.read_sql_query(percentage_size_permonth_category_q, conn)
print(total_per_size)

  pizza_size  total_sales  percentage_sales
0          L    375318.70         45.890333
1          M    249382.25         30.492044
2          S    178076.50         21.773468
3         XL     14076.00          1.721077
4        XXL      1006.60          0.123077


In [124]:
total_per_pizza_q = """ SELECT pizza_name, SUM(total_price) AS total_per_pizza FROM pizza_sales GROUP BY pizza_name
ORDER BY total_per_pizza DESC LIMIT 5;"""

total_per_pizza = pd.read_sql_query(total_per_pizza_q, conn)
print(total_per_pizza)

                     pizza_name  total_per_pizza
0        The Thai Chicken Pizza         43434.25
1    The Barbecue Chicken Pizza         42768.00
2  The California Chicken Pizza         41409.50
3      The Classic Deluxe Pizza         38180.50
4       The Spicy Italian Pizza         34831.25


In [125]:
total_per_pizza_q = """ SELECT pizza_name, SUM(total_price) AS total_per_pizza FROM pizza_sales GROUP BY pizza_name
ORDER BY total_per_pizza ASC LIMIT 5;"""

total_per_pizza = pd.read_sql_query(total_per_pizza_q, conn)
print(total_per_pizza)

                  pizza_name  total_per_pizza
0       The Brie Carre Pizza         11588.50
1     The Green Garden Pizza         13955.75
2  The Spinach Supreme Pizza         15277.75
3    The Mediterranean Pizza         15360.50
4    The Spinach Pesto Pizza         15596.00


In [128]:
quantity_q = """ SELECT pizza_name, SUM(quantity) AS quantity_per_pizza FROM pizza_sales GROUP BY pizza_name
ORDER BY  pizza_name DESC LIMIT 5;"""

quantity_per_pizza  = pd.read_sql_query(quantity_q, conn)
print(quantity_per_pizza)

                          pizza_name  quantity_per_pizza
0  The Vegetables + Vegetables Pizza                1526
1             The Thai Chicken Pizza                2371
2         The Spinach and Feta Pizza                1446
3          The Spinach Supreme Pizza                 950
4            The Spinach Pesto Pizza                 970


In [135]:
quantity_id_q = """ SELECT pizza_name, COUNT(DISTINCT order_id) AS total_orders FROM pizza_sales GROUP BY pizza_name
ORDER BY  total_orders DESC LIMIT 5;"""

total_orders  = pd.read_sql_query(quantity_id_q, conn)
print(total_orders)

                   pizza_name  total_orders
0    The Classic Deluxe Pizza          2329
1          The Hawaiian Pizza          2280
2         The Pepperoni Pizza          2278
3  The Barbecue Chicken Pizza          2273
4      The Thai Chicken Pizza          2225
